In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch
import pandas as pd

model_name = 'tiedaar/short-answer-llama'

device_map = {"": 0}
model_m = AutoPeftModelForCausalLM.from_pretrained(model_name, device_map=device_map, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#preprocessing
dataframe_path = '../data/aqag-chatgpt-vicuna.csv'

def clean_df(dataframe_path):
    df = pd.read_csv(dataframe_path)[['clean_text', 'question', 'correct_answer', 'incorrect_answer']]
    df.columns = ['source_text', 'question_text', 'True', 'False']
    df = pd.melt(df, id_vars=['source_text', 'question_text'], value_vars=['True', 'False'], var_name='labels', value_name='answer_text', col_level=None, ignore_index=True)
    df['answer_text'] = df['answer_text'].apply(lambda x: str(x))
    df['comp'] = "### Instructions \nYou will be provided with a text, a question, and an answer. Your task is to decide whether the answer is correct or incorrect and respond with a single word denoting the appropriate label: `False` for incorrect and `True` for correct.\n\n\n###The text\nThe text is as follows, delimited by three backticks: ```" + df['source_text'] + "...```\n\n\n ### The question and answer pair \n The question and answer pair are as follows, delimited by three backticks: ```##Question: " + df["question_text"] + " ##Answer: " + df['answer_text'] + "``` "
    df['text'] = df['comp'] + "\n\n\n<<<" + df['labels'] + ">>>"
    df = df[['comp', 'labels', 'text']]
    return df.reset_index()
    return df

df = clean_df(dataframe_path)

In [ ]:
from tqdm import tqdm
tqdm.pandas()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
short_df = df.head()

def make_preds(comp):
    try:
        inputs = tokenizer(comp, return_tensors='pt').to(device)
        outputs = model_m.generate(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'], max_new_tokens=1700, pad_token_id=tokenizer.eos_token_id)
        res = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return res
    except:
        print(comp)

short_df['raw_results'] = short_df['comp'].progress_apply(make_preds)

In [ ]:
df.to_csv('temp_df.csv')